In [1]:
import time
from selenium.webdriver import Firefox
import pandas as pd

In [2]:
driver = Firefox()

In [3]:
driver.get('https://www.fullertonfund.com/fund/fullerton-sgd-cash-fund-a/')

In [4]:
time.sleep(0.5)
driver.find_element(by='class name', value='button-close').click()

In [5]:
rows = []

In [6]:
for i in range(364):
    rows.extend(driver.find_element('id', 'fund-range-history').text.split('\n')[1:])
    driver.find_element('id', 'fund-range-history_next').click()
rows.extend(driver.find_element('id', 'fund-range-history').text.split('\n')[1:])

In [7]:
df = pd.DataFrame(pd.Series(rows[::-1]).str.split().apply(lambda x: x[:2]).to_list()).set_axis(['date', 'NAV'], axis=1)

In [8]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [9]:
df['NAV'] = df['NAV'].replace('\$', '', regex=True).astype(float)

In [10]:
df.set_index('date').to_csv('data/Fullerton SGD Cash Fund A.csv')

In [11]:
driver.quit()